#Web Scraping top topic from gethub
##What we will extract:
- 1  **username**
- 2  **repostire's name**
- 3  **number of stars**
- 4  **url**


###We will also extract topic name to store it in csv file that has tha same name as the topic name



In [194]:
#import libraries
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
from bs4 import BeautifulSoup
import re

In [248]:
#topics page url
MAIN_URL = "https://github.com/topics"

In [196]:
#get the html of the page
url = requests.get(MAIN_URL)
soup = BeautifulSoup(url.text, "html.parser")

In [235]:
#preproces the html to extract what we need

username_list = []
repositry_name_list = []
rating_list = []
url_list = []
for j in soup.find_all("div", {"class": "py-4 border-bottom d-flex flex-justify-between"}):
  topic_name = re.split("> <" ,j.find_all("p")[0].text)[0]
  url_sec = "https://github.com" +j.find("a").get("href")
  url_test = requests.get(url_sec)
  s = BeautifulSoup(url_test.text, "html.parser")
  with open(f"{topic_name}.csv",  "w") as f:
    writer = csv.writer(f)
    writer.writerow(["UserName","Repositry name","Rating","Url"])
  for entity in s.find_all("div", {"class": "d-flex flex-justify-between flex-items-start flex-wrap gap-2 my-3"}):

    username = entity.find("h3").find_all("a")[1].get("href").split("/")[1]
    repositry_name = entity.find("h3").find_all("a")[1].get("href").split("/")[2]
    rating = entity.find_all("span")[-1].get("aria-label").split(" ")[0]
    url = "https://github.com" + entity.find("h3").find_all("a")[1].get("href")

    with open(f"{topic_name}.csv",  "a+") as f:
          writer = csv.writer(f)
          writer.writerow([username,repositry_name,rating,url])


    username_list.append(entity.find("h3").find_all("a")[1].get("href").split("/")[1])
    repositry_name_list.append(entity.find("h3").find_all("a")[1].get("href").split("/")[2])
    rating_list.append(entity.find_all("span")[-1].get("aria-label").split(" ")[0])
    url_list.append("https://github.com" + entity.find("h3").find_all("a")[1].get("href"))


##Combining all the *csv* Files into one file called **Github Topics with their top 20 repositories.csv**

In [237]:
#extract topics names
topic_names = [re.split("> <" ,i.find_all("p")[0].text)[0] for i in soup.find_all("div", {"class": "py-4 border-bottom d-flex flex-justify-between"})]
topic_names

['3D',
 'Ajax',
 'Algorithm',
 'Amp',
 'Android',
 'Angular',
 'Ansible',
 'API',
 'Arduino',
 'ASP.NET',
 'Atom',
 'Awesome Lists',
 'Amazon Web Services',
 'Azure',
 'Babel',
 'Bash',
 'Bitcoin',
 'Bootstrap',
 'Bot',
 'C',
 'Chrome',
 'Chrome extension',
 'Command line interface',
 'Clojure',
 'Code quality',
 'Code review',
 'Compiler',
 'Continuous integration',
 'COVID-19',
 'C++']

In [259]:
#combining all the csv files into one file
combiened_data = pd.DataFrame(columns=["UserName","Repositry name","Rating","Url","Topic"])
for i in topic_names:
  data = pd.read_csv(f"{i}.csv")
  combiened_data = pd.concat([combiened_data,data], axis=0)
  combiened_data = combiened_data.fillna(i)

In [260]:
#save the csv file
combiened_data.reset_index(inplace=True, drop=True)
combiened_data.to_csv("Github Topics with their top 20 repositories.csv")